In [16]:
import os
import json

repo_path = "../refactoring-data/spring-startup-analyzer/"

# Get all the files in the repository
count = 0
llm = 0
eval = 0
for filename in os.listdir(repo_path):
    file_path = os.path.join(repo_path, filename)

    if os.path.isfile(file_path) and file_path[-5:] == ".json":
        try:
            with open(file_path) as f:
                data = json.load(f)

            if "LLMRefactoringGenerated" not in data:
                # print(f"Not generated LLM responces {filename}...")
                llm += 1
        
            if "evaluation" not in data:
                # print(f"Not evaluated {filename}...")
                eval += 1
                
            count += 1
        except Exception as e:
            print(f"Error reading {file_path}")
            print(e)
            continue

        

print("There are", count, "json files in the repository")
print("There are", llm, "files without LLM responces")
print("There are", eval, "files without evaluation")

There are 94 json files in the repository
There are 0 files without LLM responces
There are 2 files without evaluation


In [4]:
# HOW MANY FILES HAVE LLM REFACTORING AND EVALUATION IN THE WHOLE REPOSITORY
import os
import json

path = "../refactoring-data/"
total_count = 0
total_llm = 0
total_eval = 0
total_eval_error = 0
total_corrupted = 0
# Get all the files in the repository
for folder in os.listdir(path):
    repo_path = path + folder + "/"
    if not os.path.isdir(repo_path):
        continue
    count = 0
    llm = 0
    eval = 0
    eval_error = 0
    corrupted = 0
    for filename in os.listdir(repo_path):
        file_path = os.path.join(repo_path, filename)

        if os.path.isfile(file_path) and file_path[-5:] == ".json":
            try:
                with open(file_path) as f:
                    data = json.load(f)

                if "LLMRefactoringGenerated" in data:
                    # print(f"Not generated LLM responces {filename}...")
                    llm += 1
            
                if "evaluation" in data:
                    # print(f"Not evaluated {filename}...")
                    if "error" in data["evaluation"]:
                        eval_error += 1
                    else:
                        eval += 1
                if "LLMRefactoring" in data and "simplePromptTooLong" in data["LLMRefactoring"]:
                    corrupted += 1
                    
                count += 1
            except:
                print(f"Error reading {file_path}...")
                continue
    total_count += count
    total_llm += llm
    total_eval += eval
    total_eval_error += eval_error
    total_corrupted += corrupted
    #if eval != 0:
    #print(f"{folder}, Total count = {count}, LLM = {llm}, Eval = {eval}, Eval with error = {eval_error}, Corrupted = {corrupted}")
        #pass

print("Total count = ", total_count)
print("Total LLM = ", total_llm)
print("Total Eval = ", total_eval)
print("Total Eval with error = ", total_eval_error )
print("Total Corrupted = ", total_corrupted)

Error reading ../refactoring-data/PandoraNext-TokensTool/871f7426a9594b424b0bae4da76172bcc16d9596-0.json...
Total count =  2161
Total LLM =  2067
Total Eval =  1739
Total Eval with error =  0
Total Corrupted =  288


In [17]:
# Delete the error evaluation node in repo files

def delete_error_evaluation_node(path):
    count = 0
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)

        if os.path.isfile(file_path) and file_path[-5:] == ".json":
            try:
                with open(file_path) as f:
                    data = json.load(f)
                    if "evaluation" in data and "error" in data["evaluation"]:
                        del data["evaluation"]
                        with open(file_path, 'w') as f:
                            json.dump(data, f, indent=4)
                        print("Deleted evaluation node in", file_path)
                        count += 1
            except:
                print(f"Error reading {file_path}...")
                continue

    print("Deleted ERROR evaluation node in", count, "files in the repository", path)
    return count


# Delete the error evaluation node in all repo files
path = "../refactoring-data/"
c = 0
for folder in os.listdir(path):
    repo_path = path + folder + "/"
    if not os.path.isdir(repo_path):
        continue
    c += delete_error_evaluation_node(repo_path)
print(F"Deleted {c} ERROR evaluation node in files in the repository")


Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/A-tutorial-compiler-written-in-Java/
Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/HeliBoard/
Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/LiTiaotiao-Custom-Rules/
Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/pinduoduo_backdoor_unpacker/
Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/chatgpt-java/
Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/oms-erp/
Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/Logout4Shell/
Deleted ERROR evaluation node in 0 files in the repository ../refactoring-data/LakeSoul/
Deleted evaluation node in ../refactoring-data/incubator-fury/bdada2d4964dbe233ea1f3ae5ed3d362c357f899-1.json
Deleted ERROR evaluation node in 1 files in the repository ../refactoring-data/incubator-fury/
Deleted evalua

In [53]:
# Fix the LLMrefactoring format
path = "../refactoring-data/spring-startup-analyzer/"

def fixPromptFormatInRepo(path: str):
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)

        if os.path.isfile(file_path) and file_path[-5:] == ".json":
            try:
                with open(file_path) as f:
                    data = json.load(f)
                    if "LLMRefactoringGenerated" in data:
                        if "LLMRefactoring" in data and "simplePrompt" in data["LLMRefactoring"]:
                            refactoring = data["LLMRefactoring"]["simplePrompt"]
                            if refactoring[-3:] != "```":
                                if refactoring.count("`") < 6:
                                    if "simplePromptTooLong" not in data["LLMRefactoring"]:
                                        print("The LLMRefactoring too long in: ", file_path)
                                        data["LLMRefactoring"]["simplePromptTooLong"] = True
                                        with open(file_path, 'w') as f:
                                            json.dump(data, f, indent=4)
                                    # else:
                                    #     print("The LLMRefactoring too long in: ", file_path)
                                elif refactoring[-5:-2] == "```":
                                    data["LLMRefactoring"]["simplePrompt"] = refactoring[:-2]
                                    print("Fixed extra spaces in: ", file_path)
                                    with open(file_path, 'w') as f:
                                        json.dump(data, f, indent=4)
                                else:
                                    print("Problem with: ", file_path)
            except:
                print(f"Error reading {file_path}...")
                # os.remove(file_path)
                continue

In [55]:
path = "../refactoring-data/"

for folder in os.listdir(path):
    repo_path = path + folder + "/"
    if not os.path.isdir(repo_path):
        continue
    fixPromptFormatInRepo(repo_path)

Problem with:  ../refactoring-data/LiTiaotiao-Custom-Rules/e9eb1068eafadbcde61cea7bc06d55b97898b991-0.json
Problem with:  ../refactoring-data/LakeSoul/cf66a97503edd052d35c3049f6bccce78cfeef0a-9.json
Problem with:  ../refactoring-data/LakeSoul/cf66a97503edd052d35c3049f6bccce78cfeef0a-6.json
Problem with:  ../refactoring-data/LakeSoul/eb89842e472cf0d186ff091be37c6ec1c3afae1f-8.json
Problem with:  ../refactoring-data/incubator-fury/e8ff75a728c0cc57191c4db27be1c3b960b70ad7-32.json
